The base class representation of a Graph with all the interface methods

In [1]:
import abc

import numpy as np

class Graph(abc.ABC):
    def __init__(self, numVertices, directed=False):
        self.numVertices = numVertices
        self.directed = directed
        
    @abc.abstractmethod
    def add_edge(self, v1, v2, weight):
        pass
    
    @abc.abstractmethod
    def get_adjacent_vertices(self, v):
        pass
    
    @abc.abstractmethod
    def get_indegree(self, v):
        pass
    
    @abc.abstractmethod
    def get_edge_weight(self, v1, v2):
        pass
    
    @abc.abstractmethod
    def display(self):
        pass

Represents a graph as an adjacency matrix

In [3]:
class AdjacencyMatrixGraph(Graph):
    def __init__(self, numVertices, directed=False):
        super(AdjacencyMatrixGraph, self).__init__(numVertices, directed)
        
        self.matrix = np.zeros((numVertices, numVertices))
        
        
    def add_edge(self, v1, v2, weight=1):
        if v1 >= self.numVertices or v2 >= self.numVertices or v1 < 0 or v2 < 0:
            raise ValueError("Vertices %d and %d are out of bounds" % (v1, v2))
            
        if weight < 1:
            raise ValueError("An edge cannot have weight < 1")
            
        self.matrix[v1][v2] = weight
        
        if self.directed == False:
            self.matrix[v2][v1] = weight
            
            
    def get_adjacent_vertices(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Cannot access vertex %d" % v)
            
        adjacent_vertices = []
        for i in range(self.numVertices):
            if self.matrix[v][i] > 0:
                adjacent_vertices.append(i)
                
        return adjacent_vertices
    
    
    def get_indegree(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Cannot access vertex %d" % v)
            
        indegree = 0
        for i in range(self.numVertices):
            if self.matrix[i][v] > 0:
                indegree = indegree + 1
                
        return indegree
    
    
    def get_edge_weight(self, v1, v2):
        return self.matrix[v1][v2]
    
    
    def display(self):
        for i in range(self.numVertices):
            for v in self.get_adjacent_vertices(i):
                print(i, "-->", v)

Test graph as adjacency list

In [4]:
g = AdjacencyMatrixGraph(4, directed=True)

g.add_edge(0, 1)
g.add_edge(0, 2)
g.add_edge(2, 3)

for i in range(4):
    print("Adjacent to: ", i, g.get_adjacent_vertices(i))

for i in range(4):
    print("Indegree: ", i, g.get_indegree(i))

for i in range(4):
    for j in g.get_adjacent_vertices(i):
        print("Edge weight: ", i, "", j, " weight: ", g.get_edge_weight(i, j))
    
g.display()

Adjacent to:  0 [1, 2]
Adjacent to:  1 []
Adjacent to:  2 [3]
Adjacent to:  3 []
Indegree:  0 0
Indegree:  1 1
Indegree:  2 1
Indegree:  3 1
Edge weight:  0  1  weight:  1.0
Edge weight:  0  2  weight:  1.0
Edge weight:  2  3  weight:  1.0
0 --> 1
0 --> 2
2 --> 3


Represents a node

In [5]:
class Node:
    def __init__(self, vertexId):
        self.vertexId = vertexId
        self.adjacency_set = set()
        
    def add_edge(self, v):
        if self.vertexId == v:
            raise ValueError("The vertex %d cannot be adjacent to itself" % v)
            
        self.adjacency_set.add(v)
        
    def get_adjacent_vertices(self):
        return sorted(self.adjacency_set)

Represents a graph using an adjacency set

In [7]:
class AdjacencySetGraph(Graph):
    def __init__(self, numVertices, directed=False):
        super(AdjacencySetGraph, self).__init__(numVertices, directed)
        
        self.vertex_list = []
        for i in range(numVertices):
            self.vertex_list.append(Node(i))
            
            
    def add_edge(self, v1, v2, weight=1):
        if v1 >= self.numVertices or v2 >= self.numVertices or v1 < 0 or v2 < 0:
            raise ValueError("Vertices %d and %d are out of bounds" % (v1, v2))
            
        if weight != 1:
            raise ValueError("An adjacency set cannot represent edge weights > 1")
            
        self.vertex_list[v1].add_edge(v2)
        
        if self.directed == False:
            self.vertex_list[v2].add_edge(v1)


    def get_adjacent_vertices(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Cannot access vertex %d" % v)
            
        return self.vertex_list[v].get_adjacent_vertices()


    def get_indegree(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Cannot access vertex %d" % v)
            
        indegree = 0
        for i in range(self.numVertices):
            if v in self.get_adjacent_vertices(i):
                indegree = indegree + 1
                
        return indegree


    def get_edge_weight(self, v1, v2):
        return 1
    
    
    def display(self):
        for i in range(self.numVertices):
            for v in self.get_adjacent_vertices(i):
                print(i, "-->", v)

Test graph as adjacency list

In [8]:
g = AdjacencySetGraph(4, directed=True)

g.add_edge(0, 1)
g.add_edge(0, 2)
g.add_edge(2, 3)

for i in range(4):
    print("Adjacent to: ", i, g.get_adjacent_vertices(i))

for i in range(4):
    print("Indegree: ", i, g.get_indegree(i))

for i in range(4):
    for j in g.get_adjacent_vertices(i):
        print("Edge weight: ", i, "", j, " weight: ", g.get_edge_weight(i, j))
    
g.display()

Adjacent to:  0 [1, 2]
Adjacent to:  1 []
Adjacent to:  2 [3]
Adjacent to:  3 []
Indegree:  0 0
Indegree:  1 1
Indegree:  2 1
Indegree:  3 1
Edge weight:  0  1  weight:  1
Edge weight:  0  2  weight:  1
Edge weight:  2  3  weight:  1
0 --> 1
0 --> 2
2 --> 3


### Breadth-first traverstal of a graph

In [9]:
from queue import Queue

def breadth_first(graph, start=0):
    queue = Queue() # FIFO
    queue.put(start)
    
    visited = np.zeros(graph.numVertices)
    
    while not queue.empty():
        vertex = queue.get()
        
        if visited[vertex] == 1:
            continue
            
        print("Visit: ", vertex)
        visited[vertex] = 1
        
        for v in graph.get_adjacent_vertices(vertex):
            if visited[v] != 1:
                queue.put(v)
    

Test Breadth-first traversal

In [11]:
g = AdjacencyMatrixGraph(9)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(2, 7)
g.add_edge(2, 4)
g.add_edge(2, 3)
g.add_edge(1, 5)
g.add_edge(5, 6)
g.add_edge(6, 3)
g.add_edge(3, 4)
g.add_edge(6, 8)

breadth_first(g, 2)

Visit:  2
Visit:  1
Visit:  3
Visit:  4
Visit:  7
Visit:  0
Visit:  5
Visit:  6
Visit:  8


### Depth-first traversal of a graph (with recursion)

In [12]:
def depth_first(graph, visited, current=0):
    if visited[current] == 1:
        return
    
    visited[current] = 1
    print("Visit: ", current)
    
    for vertex in graph.get_adjacent_vertices(current):
        depth_first(graph, visited, vertex)

Test Depth-first traversal

In [13]:
g = AdjacencyMatrixGraph(9)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(2, 7)
g.add_edge(2, 4)
g.add_edge(2, 3)
g.add_edge(1, 5)
g.add_edge(5, 6)
g.add_edge(6, 3)
g.add_edge(3, 4)
g.add_edge(6, 8)

visited = np.zeros(g.numVertices)
depth_first(g, visited)

Visit:  0
Visit:  1
Visit:  2
Visit:  3
Visit:  4
Visit:  6
Visit:  5
Visit:  8
Visit:  7


### A Topological Sort is any ordering of all the DAG's vertices that satisfies all precedence relationships (establishing precedence)


In [16]:
from queue import Queue

def topological_sort(graph):
    queue = Queue()
    indegreeMap = {}
    
    for i in range(graph.numVertices):
        indegreeMap[i] = graph.get_indegree(i)
        
        # Queue all nodes which have no dependencies i.e. -> no edges coming in
        if indegreeMap[i] == 0:
            queue.put(i)
            
    sortedList = []
    while not queue.empty():
        vertex = queue.get()
        
        sortedList.append(vertex)
        
        for v in graph.get_adjacent_vertices(vertex):
            indegreeMap[v] = indegreeMap[v] - 1
            
            if indegreeMap[v] == 0:
                queue.put(v)
                
    if len(sortedList) != graph.numVertices:
        raise ValueError("This graph has a cycle!")
        
    print(sortedList)
    

Test topological sort

In [17]:
g = AdjacencyMatrixGraph(9, directed=True)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(2, 7)
g.add_edge(2, 4)
g.add_edge(2, 3)
g.add_edge(1, 5)
g.add_edge(5, 6)
g.add_edge(3, 6)
g.add_edge(3, 4)
g.add_edge(6, 8)

topological_sort(g)

[0, 1, 2, 5, 3, 7, 4, 6, 8]


### Shortest Path Algorithm (getting from point A to point B)

In [46]:
from queue import Queue

def build_distance_table(graph, source):
    # A dictionary mapping from the vertex number to a tuple of
    # (distance from source, last vertex on path from source)
    distance_table = {}
    
    for i in range(graph.numVertices):
        distance_table[i] = (None, None)
        
    # The distance to the source from itself is 0
    distance_table[source] = (0, source)
    
    queue = Queue()
    queue.put(source)
    
    while not queue.empty():
        current_vertex = queue.get()
        
        # The distance of the current vertex from the source
        current_distance = distance_table[current_vertex][0]
        
        for neighbor in graph.get_adjacent_vertices(current_vertex):
            # Only update the distance table if no current distance from the source is set
            if distance_table[neighbor][0] is None:
                distance_table[neighbor] = (1 + current_distance, current_vertex)
                
                # Enqueue the neighbor only if it has other adjacent vertices to explore
                if len(graph.get_adjacent_vertices(neighbor)) > 0:
                    queue.put(neighbor)
                    
    return distance_table


def shortest_path(graph, source, destination):
    distance_table = build_distance_table(graph, source)
    
    path = [destination]
    
    previous_vertex = distance_table[destination][1]
    
    while previous_vertex is not None and previous_vertex is not source:
        path = [previous_vertex] + path
        
        previous_vertex = distance_table[previous_vertex][1]
        
    if previous_vertex is None:
        print("There is no path from %d to %d" % (source, destination))
    else:
        path = [source] + path
        print("Shortest path is: ", path)

Test shortest path algorithm (unweighted graph)

In [51]:
g = AdjacencySetGraph(8, directed=True)
g.add_edge(0, 1)
g.add_edge(1, 2)
g.add_edge(1, 3)
g.add_edge(2, 3)
g.add_edge(1, 4)
g.add_edge(3, 5)
g.add_edge(5, 4)
g.add_edge(3, 6)
g.add_edge(6, 7)
g.add_edge(0, 7)

shortest_path(g, 0, 5)
shortest_path(g, 0, 6)
shortest_path(g, 7, 4)

Shortest path is:  [0, 1, 3, 5]
Shortest path is:  [0, 1, 3, 6]
There is no path from 7 to 4


### Djikstra's Algorithm

In [61]:
# https://code.activestate.com/recipes/522995-priority-dict-a-priority-queue-with-updatable-prio/
from heapq import heapify, heappush, heappop

class priority_dict(dict):
    """Dictionary that can be used as a priority queue.

    Keys of the dictionary are items to be put into the queue, and values
    are their respective priorities. All dictionary methods work as expected.
    The advantage over a standard heapq-based priority queue is
    that priorities of items can be efficiently updated (amortized O(1))
    using code as 'thedict[item] = new_priority.'

    The 'smallest' method can be used to return the object with lowest
    priority, and 'pop_smallest' also removes it.

    The 'sorted_iter' method provides a destructive sorted iterator.
    """
    
    def __init__(self, *args, **kwargs):
        super(priority_dict, self).__init__(*args, **kwargs)
        self._rebuild_heap()

    def _rebuild_heap(self):
        self._heap = [(v, k) for k, v in self.items()]
        heapify(self._heap)

    def smallest(self):
        """Return the item with the lowest priority.

        Raises IndexError if the object is empty.
        """
        
        heap = self._heap
        v, k = heap[0]
        while k not in self or self[k] != v:
            heappop(heap)
            v, k = heap[0]
        return k

    def pop_smallest(self):
        """Return the item with the lowest priority and remove it.

        Raises IndexError if the object is empty.
        """
        
        heap = self._heap
        v, k = heappop(heap)
        while k not in self or self[k] != v:
            v, k = heappop(heap)
        del self[k]
        return k

    def __setitem__(self, key, val):
        # We are not going to remove the previous value from the heap,
        # since this would have a cost O(n).
        
        super(priority_dict, self).__setitem__(key, val)
        
        if len(self._heap) < 2 * len(self):
            heappush(self._heap, (val, key))
        else:
            # When the heap grows larger than 2 * len(self), we rebuild it
            # from scratch to avoid wasting too much memory.
            self._rebuild_heap()

    def setdefault(self, key, val):
        if key not in self:
            self[key] = val
            return val
        return self[key]

    def update(self, *args, **kwargs):
        # Reimplementing dict.update is tricky -- see e.g.
        # http://mail.python.org/pipermail/python-ideas/2007-May/000744.html
        # We just rebuild the heap from scratch after passing to super.
        
        super(priority_dict, self).update(*args, **kwargs)
        self._rebuild_heap()

    def sorted_iter(self):
        """Sorted iterator of the priority dictionary items.

        Beware: this will des as they are returned.
        """
        
        while self:
            yield self.pop_smallest()

In [62]:
def dijkstra_distance_table(graph, source):
    # A dictionary mapping from the vertex number to a tuple of
    # (distance from source, last vertex on path from source)
    distance_table = {}
    
    for i in range(graph.numVertices):
        distance_table[i] = (None, None)
 
    # The distance to the source from itself is 0
    distance_table[source] = (0, source)
    
    # Holds mapping of vertex id to distance from source
    # Access the highest priority (lowest distance) item first
    priority_queue = priority_dict()
    
    priority_queue[source] = 0
    
    while len(priority_queue.keys()) > 0:
        current_vertex = priority_queue.pop_smallest()
        
        # The distance of the current node from the source
        current_distance = distance_table[current_vertex][0]
        
        for neighbor in graph.get_adjacent_vertices(current_vertex):
            distance = current_distance + g.get_edge_weight(current_vertex, neighbor)
            
            # The last recorder distance of this neighbor from the source
            neighbor_distance = distance_table[neighbor][0]
            
            # If there is a currently recorded distance from the source and this is greater than
            # the distance of the new path found, update the current distance from the source in the 
            # distance table
            if neighbor_distance is None or neighbor_distance > distance:
                distance_table[neighbor] = (distance, current_vertex)
                
                priority_queue[neighbor] = distance
                
    return distance_table

def dijkstra_shortest_path(graph, source, destination):
    distance_table = dijkstra_distance_table(graph, source)
    
    path = [destination]
    
    previous_vertex = distance_table[destination][1]
    
    while previous_vertex is not None and previous_vertex is not source:
        path = [previous_vertex] + path
        
        previous_vertex = distance_table[previous_vertex][1]
        
    if previous_vertex is None:
        print("There is no path from %d to %d" % (source, destination))
    else:
        path = [source] + path
        print("Shortest path is: ", path)

Test Dijkstra shortest path

In [64]:
g = AdjacencyMatrixGraph(8, directed=True)
g.add_edge(0, 1, 1)
g.add_edge(1, 2, 2)
g.add_edge(1, 3, 6)
g.add_edge(2, 3, 2)
g.add_edge(1, 4, 3)
g.add_edge(3, 5, 1)
g.add_edge(5, 4, 5)
g.add_edge(3, 6, 1)
g.add_edge(6, 7, 1)
g.add_edge(0, 7, 8)

dijkstra_shortest_path(g, 0, 6)
dijkstra_shortest_path(g, 4, 7)
dijkstra_shortest_path(g, 7, 0)

Shortest path is:  [0, 1, 2, 3, 6]
There is no path from 4 to 7
There is no path from 7 to 0


### Covering all nodes in a graph (minimum spanning tree) - ALL nodes are connected with maximum one edge (a spanning tree cannot have any cycles)

Given n points, connect them in the cheapest possible way so that there will be a path between every pair of points

- Minimum Spanning Tree of a Graph  ->  Spanning tree with the lowest weight

In [20]:
# Prim's Algorithm
    # works only with connected graphs
    # every intermediary graph is also a spanning tree
    # Binary Heap -> O(Eln(V ))
    # Array       -> O(E + V^2)

def spanning_tree(graph, source):
    
    # A dictionary mapping from the vertex number to a tuple of
    # (distance from source, last vertex on path from source)
    distance_table = {}
    
    for i in range(graph.numVertices):
        distance_table[i] = (None, None)
        
    # The distance to the source from itself is 0
    distance_table[source] = (0, source)
    
    # Holds mapping of vertex id to distance from source
    # Access the highest priority (lowest distance) item
    # first
    priority_queue = priority_dict()
    priority_queue[source] = 0
    
    visited_vertices = set()
    
    # Set of edges where each edge is represented as a string
    # "1->2": is an edge between vertices 1 and 2
    spanning_tree = set()
    
    while len(priority_queue.keys()) > 0:
        current_vertex = priority_queue.pop_smallest()
        
        # If we've visited a vertex earlier then we have all
        # outbound edges from it, we do not process it again
        if current_vertex in visited_vertices:
            continue
        
        visited_vertices.add(current_vertex)
        
        # If the current vertex is the source, we havent't traversed an
        # edge yet, no edge to add to our spanning tree
        if current_vertex != source:
            # The current vertex is connected by the lowest weighted edge
            last_vertex = distance_table[current_vertex][1]
            
            edge = str(last_vertex) + "-->" + str(current_vertex)
            
            if edge not in spannint_tree:
                spanning_tree.add(edge)
                
        for neighbor in graph.get_adjacent_vertices(current_vertex):
            # The distance to the neighbor is only the weight of the edge
            # connecting the neighbor
            distance = g.get_edge_weight(current_vertex, neighbor)
            
            # The last recorded distance of this neighbor
            neighbor_distance = distance_table[neighbor][0]
            
            # If this neighbor has been seen for the first time or the new edge
            # connecting this neighbor is of a lower weight than the last
            if neighbor_distance is None or neighbor_distance > distance:
                distance_table[neighbor] = (distance, current_vertex)
                priority_queue[neighbor] = distance
                
    for edge in spanning_tree:
        print(edge)
        